### Experiment with MongoEngine

The ODM we use to access MongoDB

In [159]:
from dexter.DB import DB, Account, Entry, Transaction, Document

from datetime import date

Open the database:

In [160]:
DB.open('pytest')

Make an account:

In [161]:
acct = Account(name='equity', category='Q')

Save it:

In [162]:
acct.save()

<Account: <Acct equity Q>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [163]:
Account.objects

[<Account: <Acct equity Q>, <Account: <Acct income:yoyodyne I>, <Account: <Acct bank:checking A>, <Account: <Acct amex:blue L>, <Account: <Acct chase:visa L>, <Account: <Acct expenses:groceries E>, <Account: <Acct expenses:household E>, <Account: <Acct expenses:mortgage E>, <Account: <Acct expenses:car E>, <Account: <Acct expenses:travel E>, <Account: <Acct equity Q>]

In [164]:
Account.objects[0]

<Account: <Acct equity Q>

In [165]:
acct = Account.objects[0]

In [166]:
acct.name

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [167]:
DB.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [168]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest')

In [169]:
db = DB.database

In [170]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [171]:
db.account.find_one()

{'_id': ObjectId('67ec7664cc1f8ee39d7c684a'),
 'name': 'equity',
 'category': 'Q',
 'comment': 'type: Q'}

In [172]:
for c in db.list_collections():
    print(c)

{'name': 'entry', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\x11v\x06\xaa\x85\xd5@f\x89\xebxM\xd6\xe6\x04B', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'reg_exp', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'G\xc8\xadQ\x11\xc3C\xcb\xa8\xed\xc1\xd70\xa2\xd3\x8c', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'transaction', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xdd\xca\xdb#\xf2cJ\xd3\x8c\x9b}\xfb>V\xc2&', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'account', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xe7\x1b]\xa4\xc1t@S\xa2\xb8\xa4\xf9X\xe3L\x0f', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}


In [173]:
for name in db.list_collection_names():
    print(name)

entry
reg_exp
transaction
account


In [174]:
db['account']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [175]:
db['account'].find_one()

{'_id': ObjectId('67ec7664cc1f8ee39d7c684a'),
 'name': 'equity',
 'category': 'Q',
 'comment': 'type: Q'}

In [176]:
for obj in db['account'].find():
    print(obj)

{'_id': ObjectId('67ec7664cc1f8ee39d7c684a'), 'name': 'equity', 'category': 'Q', 'comment': 'type: Q'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c684b'), 'name': 'income:yoyodyne', 'category': 'I', 'comment': 'type: I'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c684c'), 'name': 'bank:checking', 'category': 'A', 'comment': 'type: A'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c684d'), 'name': 'amex:blue', 'category': 'L', 'comment': 'type: L'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c684e'), 'name': 'chase:visa', 'category': 'L', 'comment': 'type: L'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c684f'), 'name': 'expenses:groceries', 'category': 'E', 'comment': 'type: E'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c6850'), 'name': 'expenses:household', 'category': 'E', 'comment': 'type: E'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c6851'), 'name': 'expenses:mortgage', 'category': 'E', 'comment': 'type: E'}
{'_id': ObjectId('67ec7664cc1f8ee39d7c6852'), 'name': 'expenses:car', 'category': 'E', 'comment': 'type: E'}
{'_id': O

### From Low Level to High Level

Question:  given a collection name ("account") can we find the corresponding MongoEngine class (Account)?

In [177]:
Document

mongoengine.document.Document

In [178]:
Document.__subclasses__()

[mongoengine.document.DynamicDocument,
 dexter.DB.Account,
 dexter.DB.Entry,
 dexter.DB.Transaction,
 dexter.DB.RegExp,
 __main__.Foo,
 __main__.Bar,
 __main__.FooBar]

In [179]:
[cls for cls in Document.__subclasses__() if hasattr(cls, 'objects')]

[dexter.DB.Account,
 dexter.DB.Entry,
 dexter.DB.Transaction,
 dexter.DB.RegExp,
 __main__.Foo,
 __main__.Bar,
 __main__.FooBar]

In [180]:
Account._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'account',
 'index_specs': []}

In [181]:
for cls in Document.__subclasses__():
    if not hasattr(cls, 'objects'):
        continue
    print(cls._meta['collection'], cls)

account <class 'dexter.DB.Account'>
entry <class 'dexter.DB.Entry'>
transaction <class 'dexter.DB.Transaction'>
reg_exp <class 'dexter.DB.RegExp'>
foo <class '__main__.Foo'>
bar <class '__main__.Bar'>
foo_bar <class '__main__.FooBar'>


### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [182]:
t = Transaction(description='hi', comment='aloha')

In [183]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [184]:
t.tags

[]

In [185]:
t.entries

[]

### Entries

In [186]:
e = Entry(uid='xxx', column='credit', date='2025-03-05', amount=1000, account='unknown')

In [187]:
type(e)

dexter.DB.Entry

In [188]:
e

<Entry: <En 2025-03-05 unknown -$1000.0>>

In [189]:
e.column

<Column.cr: 'credit'>

In [190]:
e.column.opposite()

<Column.dr: 'debit'>

In [191]:
e.column.opposite().opposite()

<Column.cr: 'credit'>

In [192]:
e.amount

1000.0

In [193]:
e.hash

'6fce51cdae9a1803b7c8d26e12244edc'

In [194]:
len(e.hash)

32

In [195]:
{e.uid for e in Entry.objects}

{'00b4f25403825fae3ea11e3cc1b428a7',
 '01a6c8b9c5ea155bcf6a294036ee0fef',
 '039ce819bf4be6c97c0e562021c0c291',
 '03ed18f5b5af7373f596bc130a522d31',
 '0bc92e7bd20c17a50b7e83a8fc16b213',
 '0ea0c58c6abdfb9814a5bbb3466e2be9',
 '0f7217acf9b924953cdb1b26dbe3b81b',
 '14d80a9ba5338682afe0425985ccd1ae',
 '1732e5ef3f64fc8fc28429a867b994f2',
 '18040d2b59acd3346122c3f084201dcd',
 '1d31426b51943f9a1588576ec728de4b',
 '20c32394280e4bf00f0faf05aab51d3d',
 '26b51be8b3c9104d984ff45aa06236a6',
 '34bee8ca5e3faf0497a19cf7b1e9e65c',
 '3b10be5f8ae185da6253aaefa97a586a',
 '3e687f99e70f3b52b0e5ea07f28ef887',
 '3f734f34bd78107418621a2e01ba8540',
 '529564ade155891bd2bdb23f9f776530',
 '52b081ba44cf4600a8294b8d640b270c',
 '6f2d599c07e27edc03feeb8bf2acdc80',
 '6ff8eec43005d6834a414d4f307655a9',
 '7521fde980ec531f930927810424b332',
 '8291d7bf3f1960e028944879cf5f8690',
 '842583d7002b48f2776d5b687860f4b5',
 '87c83a9615a2780e3680c684faa0f073',
 '91d6003b87cc6c92ffbce4a2f4b375f8',
 '950f17ee230abc8e769a4b8189c100b9',
 

In [196]:
s = set()
for e in Entry.objects:
    if e in s:
        print(e.date, e.amount, e.description)
    s.add(e.uid)

In [197]:
len(s)

38

In [198]:
len(Entry.objects)

38

In [199]:
lst = sorted([e.uid for e in Entry.objects])

In [200]:
len(lst)

38

In [201]:
dup = []
for i in range(len(lst)-1):
    if lst[i] == lst[i+1]:
        dup.append(lst[i])

In [202]:
dup

[]

In [203]:
for e in Entry.objects:
    if e.uid in dup:
        print(e.date, e.amount, e.account, e.description)

### Tags

In [204]:
e = Entry.objects[0]

In [205]:
e.description

'Jan pay'

In [206]:
e.tags.append('a')

In [207]:
e.save()

<Entry: <En 2024-01-02 bank:checking +$5000.0>>

In [209]:
e.tags

['a']

In [210]:
e.tags += ['b','c','d']

In [211]:
e.save()

<Entry: <En 2024-01-02 bank:checking +$5000.0>>

In [212]:
e.tags.remove('c')

In [213]:
e.save()

<Entry: <En 2024-01-02 bank:checking +$5000.0>>

In [214]:
e.note = "hello"

In [216]:
e.save()

<Entry: <En 2024-01-02 bank:checking +$5000.0>>

### References

The big test -- can we add that Entry to the transaction?

In [ ]:
t.entries.append(e)

In [ ]:
t.entries

Yes!  🎉

### Misc Commands

In [ ]:
db.stats

In [ ]:
db.stats.find_one

In [ ]:
db.list_collection_names()

In [ ]:
db.command('count','account')

In [ ]:
db.command('hello')

In [ ]:
db.command('hostInfo')

In [ ]:
db.command('ping')

### Fetch Transactions

Specify constraints on transactions

In [ ]:
Transaction.objects

In [ ]:
Transaction.objects(description='Safeway')

In [ ]:
for t in Transaction.objects(description='Safeway'):
    for e in t.entries:
        print(e.date, e.account, e.amount, e.column)

In [ ]:
for t in Transaction.objects(description='Safeway'):
    print(t.accounts)

In [ ]:
for t in Transaction.objects(description='Safeway'):
    print(t.pamount)

In [ ]:
for t in Transaction.objects(description='Safeway'):
    print(t.pdate, type(t.pdate))

In [ ]:
for t in Transaction.objects(description='Safeway'):
    print(t.originals)

In [ ]:
lst = list(Transaction.objects(description='Safeway'))

In [ ]:
lst[1].comment

In [ ]:
lst[1].pamount

In [ ]:
lst[1].pdate

In [ ]:
list(Transaction.objects(pamount__lt=175.0))

In [ ]:
for t in Transaction.objects(pdate=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

In [ ]:
for t in Transaction.objects(pdate__lte=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

### Operators

In [ ]:
for t in Transaction.objects(description__gte='Safeway'):
    print(t.pdate, t.description)

In [ ]:
for t in Transaction.objects(description__regex='^S'):
    print(t.pdate, t.description)

The operator automatically applies to list elements.

In [ ]:
for t in Transaction.objects(description__regex=r'\s'):
    print(t.pdate, t.description, t.pamount)

For compound constraints we need another class from MongoEngine.

In [ ]:
from mongoengine.queryset.visitor import Q

In [ ]:
for t in Transaction.objects(Q(description__regex=r'^S')):
    print(t.pdate, t.description)

In [ ]:
for t in Transaction.objects(Q(description__regex=r'^S') & Q(description__regex=r'\s')):
    print(t.pdate, t.description)

### QuerySet

In [ ]:
for a in Account.nominal_accounts:
    print(a.name)

### Combining Query Elements

In [ ]:
q = Q(description__regex=r'^S')

In [ ]:
q

In [ ]:
type(q)

In [ ]:
p = Q(description__regex=r'\s')

In [ ]:
p & q

In [ ]:
for t in Transaction.objects(p & q):
    print(t.pdate, t.description)

Create Q object using dictionaries

In [ ]:
dct = {'description__regex': r'^S'}

In [ ]:
Q(**dct)

Can an object have multiple constraints?

In [ ]:
dct = {'description__regex': r'^S', 'pamount__gt': 100}

In [ ]:
Q(**dct)

In [ ]:
for t in Transaction.objects(Q(**dct)):
    print(t.pdate, t.description, t.pamount)

Yep!

### Select Method

#### Select Transactions

All transactions:

In [ ]:
for t in DB.select(Transaction):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

By date:

In [ ]:
for t in DB.select(Transaction, date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, start_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, end_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

By amount:

In [ ]:
for t in DB.select(Transaction, amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
lst = DB.select(Transaction, amount=75)

In [ ]:
all(t.pamount == 75 for t in lst)

In [ ]:
for t in DB.select(Transaction, max_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, min_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

By description:

In [ ]:
for t in DB.select(Transaction, description = r'^s'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, comment=r'budget'):
    print(t.pdate, t.pamount, t.description, t.comment, t.pcredit, t.pdebit)

By account:

In [ ]:
for t in DB.select(Transaction, debit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, credit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

Some random combinations

In [ ]:
for t in DB.select(Transaction, description = r'^s', min_amount=100):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

In [ ]:
for t in DB.select(Transaction, start_date = date(2024,2,1), credit='visa'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

#### Select Entries

All entries:

In [ ]:
len(DB.select(Entry))

In [ ]:
for e in DB.select(Entry):
    print(e.date, e.account, e.amount, e.column)

By date:

In [ ]:
for e in DB.select(Entry, date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, start_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, end_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

By amount:

In [ ]:
for e in DB.select(Entry, amount=900):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, max_amount=900):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, min_amount=900):
    print(e.date, e.account, e.amount, e.column)

By account:

In [ ]:
for e in DB.select(Entry, account='groceries'):
    print(e.date, e.account, e.amount, e.column)

By column:

In [ ]:
for e in DB.select(Entry, column='credit'):
    print(e.date, e.account, e.amount, e.column)

In [ ]:
for e in DB.select(Entry, column='debit'):
    print(e.date, e.account, e.amount, e.column)

### Serializing Objects

In [ ]:
import json
from bson.objectid import ObjectId
import datetime

In [ ]:
lst = DB.select(Transaction, start_date = date(2024,2,1), credit='visa')

In [ ]:
lst[0].to_json()

In [ ]:
type(lst[0])

In [ ]:
obj = Transaction.objects.as_pymongo()[0]

In [ ]:
type(obj)

In [ ]:
obj

In [ ]:
s = lst[0].to_json()

In [ ]:
json.loads(s)

In [ ]:
Transaction.from_json(s)

In [ ]:
s = 'account: {...:...}'

In [ ]:
s.find(':')

In [ ]:
s[:s.find(':')]

In [ ]:
s[s.find(':'):]

### Indexes

We want a field in Entry documents that serves as a unique ID so we can tell if an item was imported before.

MongoEngine has a UUID field.
* how is it computed?  is it a hash of all the other field values?
* when is it computed?  when the object is made, or when it is saved?

In [ ]:
from mongoengine import *

In [ ]:
class Foo(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)

In [ ]:
f = Foo(name='Fred', amount=10)

Just declaring it is not enough to give it a value:

In [ ]:
f.uid is None

In [ ]:
f.save()

This model has an index.  The `#` means it's a "hashed index" but no discussion of what that means or why we'd want one (over say a text index that we compute ourselves).

In [ ]:
class Bar(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)
    meta = {
        'indexes': ['#uid']
    }

In [ ]:
b1 = Bar(name='george', amount=20, uid='123')

In [ ]:
b1.uid

Ah -- the UUID is created when the document is saved (the same was true for Foo, above, if a `uid` value passed to the constructor).

```
b1.save()
...
ValidationError: ValidationError (Bar:None) (Could not convert to UUID: badly formed hexadecimal UUID string: ['uid'])
```

So how do we make a UUID?  Do we care?  Why not just use our hashed strings?

#### UUID

It's in the PyMongo docs (and we had to specify how they're represented when we made the DB connection).  It's also a Python library.

In [ ]:
from uuid import uuid4

In [ ]:
b2 = Bar(name='ringo', amount=30, uid=uuid4())

In [ ]:
b2.uid

In [ ]:
b2.save()

In [ ]:
x = b2.uid

In [ ]:
b3 = Bar(name='paul', amount=40, uid=x)

In [ ]:
b3.save()

In [ ]:
b3.uid == b2.uid

In [ ]:
uuid4()

So just defining an index isn't enough to make it unique.

### Unique

In [ ]:
class FooBar(Document):
    name = StringField()
    amount = FloatField()
    uid = StringField(unique=True)

In [ ]:
f1 = FooBar(name='Fred', amount=100, uid='bedrock')

In [ ]:
f2 = FooBar(name='Barney', amount=200, uid='bedrock')

In [ ]:
f1.save()

In [ ]:
try:
    f2.save()
except NotUniqueError as err:
    print(err)

In [ ]:
FooBar._meta

Awesome!  Just defining a field as unique is enough to have MongoEngine create an index.  Don't know (and don't care, yet, at least) about the ramifications of `sparse = False`.

> checked PyMongo docs, it's not what we think, and not something we want (even though we can have it by specifying `sparse=True` in the column spec)

### Match Account Names

In [ ]:
Account.objects(name__contains='checking')

In [ ]:
Account.objects(name__contains='g')

In [ ]:
DB.find_account('g')